In [1]:
# Colab setup ------------------
import os, sys, subprocess
# ------------------------------

import numpy as np
import scipy.integrate

import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting

import GATA6_response_model

# Set to True to have fully interactive plots with Python;
# Set to False to use pre-built JavaScript-based plots
interactive_python_plots = False
notebook_url = "localhost:8888"

bokeh.io.output_notebook()

def callback(attr, old, new):
    for i in range(len(gata6)):
        _, _, ss = GATA6_response_model.RunDeterministicResponse(i * 1000,
                                                                inhibited_transcription_rate = [0,0],
                                                                activated_transcription_rate = [pFOXA2_FOXA2_tr_slider.value, pFOXA2_GATA6_tr_slider.value, pFOXF1_FOXF1_tr_slider.value, pFOXF1_GATA6_tr_slider.value],
                                                                double_activated_transcription_rate = [pFOXA2_FOXA2_GATA6_tr_slider.value, pFOXF1_FOXF1_GATA6_tr_slider.value],
                                                                P_act_binding=[FOXA2_p0_FOXA2_binding_slider.value, FOXF1_p0_FOXF1_binding_slider.value, FOXA2_p0_GATA6_binding_slider.value, FOXF1_p0_GATA6_binding_slider.value],
                                                                P_inh_binding=[FOXA2_p0_FOXA2_FOXF1_binding_slider.value, FOXF1_p0_FOXF1_FOXA2_binding_slider.value],
                                                                P1_act_binding=[FOXA2_FOXA2_p1_full_binding_slider.value, FOXF1_FOXF1_p1_full_binding_slider.value, FOXA2_GATA6_p1_full_binding_slider.value, FOXF1_GATA6_p1_full_binding_slider.value],
                                                                P1_inh_unbinding=[FOXA2_FOXF1_p1_FOXA2_unbinding_slider.value, FOXF1_FOXA2_p1_FOXF1_unbinding_slider.value],
                                                                P1_act_unbinding=[FOXA2_p1_FOXA2_unbinding_slider.value, FOXF1_p1_FOXF1_unbinding_slider.value, FOXA2_p1_GATA6_unbinding_slider.value, FOXF1_p1_GATA6_unbinding_slider.value],
                                                                P2_act_unbinding=[p2_FOXA2_FOXA2_unbinding_slider.value, p2_FOXF1_FOXF1_unbinding_slider.value, p2_FOXA2_GATA6_unbinding_slider.value, p2_FOXF1_GATA6_unbinding_slider.value])
        foxa2s[i] = ss[-1,2]
        foxf1s[i] = ss[-1,3]
    cds.data["foxa2"] = foxa2s
    cds.data["foxf1"] = foxf1s

pFOXA2_GATA6_tr_slider = bokeh.models.Slider(
    title="pFOXA2_GATA6 activated transcription rate", start=0, end=1, step=0.01, value=0.25, width=150
)
pFOXA2_FOXA2_tr_slider = bokeh.models.Slider(
    title="pFOXA2_FOXA2 activated transcription rate", start=0, end=1, step=0.01, value=0.25, width=150
)
pFOXA2_FOXA2_GATA6_tr_slider = bokeh.models.Slider(
    title="pFOXA2_GATA6_FOXA2 activated transcription rate", start=0, end=1, step=0.01, value=0.25, width=150
)
pFOXF1_GATA6_tr_slider = bokeh.models.Slider(
    title="pFOXF1_GATA6 activated transcription rate", start=0, end=1, step=0.01, value=0.25, width=150
)
pFOXF1_FOXF1_tr_slider = bokeh.models.Slider(
    title="pFOXF1_FOXF1 activated transcription rate", start=0, end=1, step=0.01, value=0.5, width=150
)
pFOXF1_FOXF1_GATA6_tr_slider = bokeh.models.Slider(
    title="pFOXF1_GATA6_FOXF1 activated transcription rate", start=0, end=1, step=0.01, value=0.01, width=150
)

FOXA2_p0_FOXA2_binding_slider = bokeh.models.Slider(
    title="FOXA2 P0 -> P1 FOXA2_FOXA2 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
FOXF1_p0_FOXF1_binding_slider = bokeh.models.Slider(
    title="FOXF1 P0 -> P1 FOXF1_FOXF1 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
FOXA2_p0_GATA6_binding_slider = bokeh.models.Slider(
    title="FOXA2 P0 -> P1 FOXA2_GATA6 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
FOXF1_p0_GATA6_binding_slider = bokeh.models.Slider(
    title="FOXF1 P0 -> P1 FOXF1_GATA6 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
FOXA2_p0_FOXA2_FOXF1_binding_slider = bokeh.models.Slider(
    title="FOXA2 P0 -> P1 FOXA2_FOXF1 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
FOXF1_p0_FOXF1_FOXA2_binding_slider = bokeh.models.Slider(
    title="FOXF1 P0 -> P1 FOXF1_FOXA2 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)

FOXA2_p1_FOXA2_unbinding_slider = bokeh.models.Slider(
    title="FOXA2_FOXA2 P1 -> P0 FOXA2 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)
FOXF1_p1_FOXF1_unbinding_slider = bokeh.models.Slider(
    title="FOXF1_FOXF1 P1 -> P0 FOXF1 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)
FOXA2_p1_GATA6_unbinding_slider = bokeh.models.Slider(
    title="FOXA2_GATA6 P1 -> P0 FOXA2 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)
FOXF1_p1_GATA6_unbinding_slider = bokeh.models.Slider(
    title="FOXF1_GATA6 P1 -> P0 FOXF1 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)

FOXA2_FOXF1_p1_FOXA2_unbinding_slider = bokeh.models.Slider(
    title="FOXA2_FOXF1 P1 -> P0 FOXA2 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)
FOXF1_FOXA2_p1_FOXF1_unbinding_slider = bokeh.models.Slider(
    title="FOXF1_FOXA2 P0 -> P1 FOXF1 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)

FOXA2_FOXA2_p1_full_binding_slider = bokeh.models.Slider(
    title="FOXA2_FOXA2 P1 -> P2 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
FOXF1_FOXF1_p1_full_binding_slider = bokeh.models.Slider(
    title="FOXF1_FOXF1 P1 -> P2 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
FOXA2_GATA6_p1_full_binding_slider = bokeh.models.Slider(
    title="FOXA2_GATA6 P1 -> P2 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
FOXF1_GATA6_p1_full_binding_slider = bokeh.models.Slider(
    title="FOXF1_GATA6 P1 -> P2 binding rate", start=0, end=1000, step=1.0, value=1.0, width=150
)
p2_FOXA2_FOXA2_unbinding_slider = bokeh.models.Slider(
    title="P2 -> P1 FOXA2_FOXA2 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)
p2_FOXF1_FOXF1_unbinding_slider = bokeh.models.Slider(
    title="P2 -> P1 FOXF1_FOXF1 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)
p2_FOXA2_GATA6_unbinding_slider = bokeh.models.Slider(
    title="P2 -> P1 FOXA2_GATA6 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)
p2_FOXF1_GATA6_unbinding_slider = bokeh.models.Slider(
    title="P2 -> P1 FOXF1_GATA6 binding rate", start=0, end=1000, step=1.0, value=100.0, width=150
)

# Set up time points
gata6_max = 50
gata6 = np.linspace(0, gata6_max, 6)
t = np.linspace(0, 10000, 10000)

foxa2s = np.zeros(len(gata6))
foxf1s = np.zeros(len(gata6))

# Build s, taking slider values are parameters.


for i in range(len(gata6)):
    _, _, ss = GATA6_response_model.RunDeterministicResponse(i * 1000,  
                                                            inhibited_transcription_rate = [0,0],
                                                            activated_transcription_rate = [pFOXA2_FOXA2_tr_slider.value, pFOXA2_GATA6_tr_slider.value, pFOXF1_FOXF1_tr_slider.value, pFOXF1_GATA6_tr_slider.value],
                                                            double_activated_transcription_rate = [pFOXA2_FOXA2_GATA6_tr_slider.value, pFOXF1_FOXF1_GATA6_tr_slider.value],
                                                            P_act_binding=[FOXA2_p0_FOXA2_binding_slider.value, FOXF1_p0_FOXF1_binding_slider.value, FOXA2_p0_GATA6_binding_slider.value, FOXF1_p0_GATA6_binding_slider.value],
                                                            P_inh_binding=[FOXA2_p0_FOXA2_FOXF1_binding_slider.value, FOXF1_p0_FOXF1_FOXA2_binding_slider.value],
                                                            P1_act_binding=[FOXA2_FOXA2_p1_full_binding_slider.value, FOXF1_FOXF1_p1_full_binding_slider.value, FOXA2_GATA6_p1_full_binding_slider.value, FOXF1_GATA6_p1_full_binding_slider.value],
                                                            P1_inh_unbinding=[FOXA2_FOXF1_p1_FOXA2_unbinding_slider.value, FOXF1_FOXA2_p1_FOXF1_unbinding_slider.value],
                                                            P1_act_unbinding=[FOXA2_p1_FOXA2_unbinding_slider.value, FOXF1_p1_FOXF1_unbinding_slider.value, FOXA2_p1_GATA6_unbinding_slider.value, FOXF1_p1_GATA6_unbinding_slider.value],
                                                            P2_act_unbinding=[p2_FOXA2_FOXA2_unbinding_slider.value, p2_FOXF1_FOXF1_unbinding_slider.value, p2_FOXA2_GATA6_unbinding_slider.value, p2_FOXF1_GATA6_unbinding_slider.value])
    foxa2s[i] = ss[-1,2]
    foxf1s[i] = ss[-1,3]


# Place the data in a ColumnDataSource
cds = bokeh.models.ColumnDataSource(dict(gata6=gata6, foxa2=foxa2s, foxf1=foxf1s))

# Build the plot
p = bokeh.plotting.figure(
    frame_height=400,
    frame_width=400,
    x_axis_label="GATA6 / 1000",
    y_axis_label="GATA6 Response",
    x_range=[0, 50],
    y_range=[-0.02, 50000],
)
p.line(source=cds, x="gata6", y="foxa2", line_width=2, color='red', legend_label="FOXA2");
p.line(source=cds, x="gata6", y="foxf1", line_width=2, color='green', legend_label="FOXF1")

pFOXA2_GATA6_tr_slider.on_change("value", callback)
pFOXA2_FOXA2_tr_slider.on_change("value", callback)
pFOXA2_FOXA2_GATA6_tr_slider.on_change("value", callback)
pFOXF1_GATA6_tr_slider.on_change("value", callback)
pFOXF1_FOXF1_tr_slider.on_change("value", callback)
pFOXF1_FOXF1_GATA6_tr_slider.on_change("value", callback)

FOXA2_p0_FOXA2_binding_slider.on_change("value", callback)
FOXF1_p0_FOXF1_binding_slider.on_change("value", callback)
FOXA2_p0_GATA6_binding_slider.on_change("value", callback)
FOXF1_p0_GATA6_binding_slider.on_change("value", callback)
FOXA2_p0_FOXA2_FOXF1_binding_slider.on_change("value", callback)
FOXF1_p0_FOXF1_FOXA2_binding_slider.on_change("value", callback)

FOXA2_p1_FOXA2_unbinding_slider.on_change("value", callback)
FOXF1_p1_FOXF1_unbinding_slider.on_change("value", callback)
FOXA2_p1_GATA6_unbinding_slider.on_change("value", callback)
FOXF1_p1_GATA6_unbinding_slider.on_change("value", callback)
FOXA2_FOXF1_p1_FOXA2_unbinding_slider.on_change("value", callback)
FOXF1_FOXA2_p1_FOXF1_unbinding_slider.on_change("value", callback)

FOXA2_FOXA2_p1_full_binding_slider.on_change("value", callback)
FOXF1_FOXF1_p1_full_binding_slider.on_change("value", callback)
FOXA2_GATA6_p1_full_binding_slider.on_change("value", callback)
FOXF1_GATA6_p1_full_binding_slider.on_change("value", callback)
p2_FOXA2_FOXA2_unbinding_slider.on_change("value", callback)
p2_FOXF1_FOXF1_unbinding_slider.on_change("value", callback)
p2_FOXA2_GATA6_unbinding_slider.on_change("value", callback)
p2_FOXF1_GATA6_unbinding_slider.on_change("value", callback)

layout = bokeh.layouts.row(
    p,
    bokeh.models.Spacer(width=30),
    bokeh.layouts.column(pFOXA2_GATA6_tr_slider,
                        pFOXA2_FOXA2_tr_slider,
                        pFOXA2_FOXA2_GATA6_tr_slider,
                        pFOXF1_GATA6_tr_slider,
                        pFOXF1_FOXF1_tr_slider,
                        pFOXF1_FOXF1_GATA6_tr_slider),
    bokeh.models.Spacer(width=200),
    bokeh.layouts.column(FOXA2_p0_FOXA2_binding_slider,
                        FOXF1_p0_FOXF1_binding_slider,
                        FOXA2_p0_GATA6_binding_slider,
                        FOXF1_p0_GATA6_binding_slider,
                        FOXA2_p0_FOXA2_FOXF1_binding_slider,
                        FOXF1_p0_FOXF1_FOXA2_binding_slider),
    bokeh.models.Spacer(width=150),
    bokeh.layouts.column(
                        FOXA2_p1_FOXA2_unbinding_slider,
                        FOXF1_p1_FOXF1_unbinding_slider,
                        FOXA2_p1_GATA6_unbinding_slider,
                        FOXF1_p1_GATA6_unbinding_slider,
                        FOXA2_FOXF1_p1_FOXA2_unbinding_slider,
                        FOXF1_FOXA2_p1_FOXF1_unbinding_slider),
    bokeh.models.Spacer(width=150),
    bokeh.layouts.column(
                        FOXA2_FOXA2_p1_full_binding_slider,
                        FOXF1_FOXF1_p1_full_binding_slider,
                        FOXA2_GATA6_p1_full_binding_slider,
                        FOXF1_GATA6_p1_full_binding_slider,
                        p2_FOXA2_FOXA2_unbinding_slider,
                        p2_FOXF1_FOXF1_unbinding_slider,
                        p2_FOXA2_GATA6_unbinding_slider,
                        p2_FOXF1_GATA6_unbinding_slider)
)

def app(doc):
    doc.add_root(layout)

Loading BokehJS ...

In [2]:
interactive_python_plots = True

if interactive_python_plots:
    bokeh.io.show(app, notebook_url=notebook_url)